In [ ]:
import torch
import torch.utils.data
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
from tqdm import tqdm 
import pickle
import time
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable
import math

In [ ]:
from tdnn import TDNN

## Statstical pooling implementation
#### implement a module that takes a matrix and calculates mean and std 
#### of each channel (y axis) and concatinates them

In [ ]:
########################
#    your code here    #
########################

 ## Data preperation

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
clases = ['Trump','Macron','Gaga','Clinton']

### Below is a code that extracts mfcc for training
#### write a code that extracts mffcc (using the correct number of features) 
#### from chunked wav and updates the global _min and _max

In [ ]:
X = []
y = []
max_total_context = 400
_min,_max = float('inf'),-float('inf')

for wav_name in os.listdir('train'):
    if wav_name.endswith('wav'):
        print (wav_name)
        rate, wav = wavfile.read(r'train/'+wav_name)
        for chunked_wav in tqdm(chunks(wav,int(len(wav)/4))):
            #######################################
            #
            #          your code here               
            #
            
            for chunked_X_sample in list(chunks(X_sample,max_total_context)):
                if len(chunked_X_sample) == max_total_context:
                    X.append(chunked_X_sample)
                    y.append(to_categorical(clases.index(wav_name.split('.wav')[0]),len(clases)))
                else:
                    print ('discarded box')
X = (X - _min) / (_max-_min)

#### is the amound of data of each speaker is equal ?
#### Make an attempt to fix it if so 

In [ ]:
plt.hist(np.argmax(y,axis=1))

In [ ]:
with open('X_data.pkl','wb') as f:
    pickle.dump((X,y),f)

In [ ]:
with open('X_data.pkl','rb') as f:
    X,y =  pickle.load(f)

In [ ]:

trainloader = torch.utils.data.DataLoader(list(zip(X,y)), shuffle=True, batch_size=50)

## Xvector implementation
#### Notice that the TDNN layer is already implemented, read the documentation in the file
#### tdnn.py, use the context in the paper we used (as printed in the slides) and build the complete Xvector 
#### architecture.
#### *Make sure you can get the embedding easly from the architecture*


In [ ]:
########################
#    your code here    #
########################

In [ ]:
net = Xvector()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
net = net.cuda()

### Train the model the model that you have implemented

In [ ]:
epocs = 300
for j in range (epocs):
    for i, data in enumerate(trainloader, 0):
        
            ########################
            #    your code here    #
            ########################
        
    print ("loss: {} step took {}".format(loss,b-a))


In [ ]:
  torch.save(net, 'model')


 ### Testing

In [ ]:
xvec = torch.load('model')

In [ ]:
xvec.eval()

In [ ]:
xvec.cuda()

In [ ]:
wav_path = "train/Gaga.wav"
max_total_context_test = 400
_min,_max = float('inf'),-float('inf')
X_test = []
rate, wav = wavfile.read(wav_path)
for chunked_wav in tqdm(list(chunks(wav,int(len(wav)/400)))[:100]):
    X_sample = mfcc(chunked_wav,samplerate= rate,numcep=24
           ,nfilt=26,nfft=1024)
    _min = min(np.amin(X_sample),_min)
    _max = max(np.amax(X_sample),_max)
    for chunked_X_sample in list(chunks(X_sample,max_total_context_test)):
            if len(chunked_X_sample) == max_total_context_test:
                X_test.append(chunked_X_sample)
X_test = (X_test - _min) / (_max-_min)


## Try the model on the data extracted above

### Extra mission
#### extract some x-vectors for the train data and apply LDA dimention reduction to it
#### plot the low dimension vectors you got